## 0. Imports & configuration

In [6]:
import re
from pathlib import Path

import numpy as np
import pandas as pd


## 1. Load & concat the 10 CSVs

In [7]:
import pandas as pd
from pathlib import Path

# 1. set the folder that contains your 10 CSV files
DATA_DIR = Path("/kaggle/input/euro-man")  # <-- change this

# 2. set a file name pattern that matches all 10 files
PATTERN = "euro*.csv"  # <-- change if your names are different

# 3. find all matching files
files = sorted(DATA_DIR.glob(PATTERN))

print("Found files:")
for f in files:
    print("  ", f.name)

print("Number of files:", len(files))


Found files:
   euro000000000000.csv
   euro000000000001.csv
   euro000000000002.csv
   euro000000000003.csv
   euro000000000004.csv
   euro000000000005.csv
   euro000000000006.csv
   euro000000000007.csv
   euro000000000008.csv
   euro000000000009.csv
Number of files: 10


In [8]:
dfs = []
for f in files:
    print("Reading", f.name)
    df_part = pd.read_csv(f, low_memory=False)
    dfs.append(df_part)

df_raw = pd.concat(dfs, ignore_index=True)
print("Combined rows:", len(df_raw))
print("Columns:", df_raw.columns.tolist())


Reading euro000000000000.csv
Reading euro000000000001.csv
Reading euro000000000002.csv
Reading euro000000000003.csv
Reading euro000000000004.csv
Reading euro000000000005.csv
Reading euro000000000006.csv
Reading euro000000000007.csv
Reading euro000000000008.csv
Reading euro000000000009.csv
Combined rows: 763728
Columns: ['publish_time', 'publish_date', 'article_url', 'source_name', 'tone_raw', 'tone_main', 'themes', 'locations', 'source_language', 'translated_to_english', 'relevance_score']


In [9]:
df_raw.to_parquet("euro_manu_news_all.parquet", index=False)

In [10]:
import pandas as pd

df = df_raw.copy()  # if you already use df, skip this line

df["publish_time"] = pd.to_datetime(df["publish_time"], errors="coerce", utc=True)
df["publish_date"] = pd.to_datetime(df["publish_date"], errors="coerce")

df["month_ts"] = (
    df["publish_date"]
    .dt.to_period("M")
    .dt.to_timestamp("M")   # month end
)


### 直接从这里开始运行

In [11]:
import pandas as pd

df_raw = pd.read_parquet("euro_manu_news_all.parquet")
print("Loaded rows:", len(df_raw))


Loaded rows: 763728


## 2. basic cleaning

In [12]:
import re
import numpy as np
import pandas as pd

df = df_raw.copy()

# Convert dates / times
df["publish_date"] = pd.to_datetime(df["publish_date"], errors="coerce")
df["publish_time"] = pd.to_datetime(df["publish_time"], errors="coerce")

# Numeric conversions
for col in ["tone_main", "relevance_score", "translated_to_english"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Drop rows with no URL or invalid tone
df = df.dropna(subset=["article_url", "tone_main"]).copy()

# Optional: restrict to time window
mask_period = (df["publish_date"] >= "2010-01-01") & (df["publish_date"] <= "2025-12-31")
df = df.loc[mask_period].copy()

print("Rows after basic cleaning:", len(df))


Rows after basic cleaning: 763728


### 去掉明显重复

In [13]:
df = df.sort_values(["article_url", "source_name", "publish_time"])

df = df.drop_duplicates(
    subset=["publish_date", "article_url", "source_name"],
    keep="first",
).reset_index(drop=True)

print("Rows after dropping duplicates:", len(df))


Rows after dropping duplicates: 763720


### 解析 GDELT 的国家信息并 explode

In [14]:
EURO_FIPS_TO_ISO2 = {
    "GM": "DE", "FR": "FR", "IT": "IT", "SP": "ES", "NL": "NL",
    "AU": "AT", "BE": "BE", "CY": "CY", "EN": "EE", "FI": "FI",
    "GR": "GR", "EI": "IE", "LG": "LV", "LH": "LT", "LU": "LU",
    "MT": "MT", "PO": "PT", "LO": "SK", "SI": "SI",
}

LOCATION_COUNTRY_RE = re.compile(r"#([A-Z]{2})#")


def extract_country_fips_list(loc: str) -> list[str]:
    if not isinstance(loc, str) or not loc.strip():
        return []
    codes = LOCATION_COUNTRY_RE.findall(loc)
    return list(dict.fromkeys(codes))  # unique, keep order


def count_locations(loc: str) -> int:
    if not isinstance(loc, str) or not loc.strip():
        return 0
    return len([x for x in loc.split(";") if x])


df["country_fips_list"] = df["locations"].apply(extract_country_fips_list)
df["location_count"] = df["locations"].apply(count_locations)

df_exploded = df.explode("country_fips_list", ignore_index=True)
df_exploded = df_exploded.rename(columns={"country_fips_list": "country_fips"})

euro_fips_set = set(EURO_FIPS_TO_ISO2.keys())
df_exploded = df_exploded[df_exploded["country_fips"].isin(euro_fips_set)].copy()

df_exploded["country_iso2"] = df_exploded["country_fips"].map(EURO_FIPS_TO_ISO2)

print("Rows after exploding:", len(df_exploded))
print("Countries:", df_exploded["country_iso2"].dropna().unique())


Rows after exploding: 1102142
Countries: ['GR' 'FR' 'CY' 'DE' 'AT' 'BE' 'IT' 'ES' 'SI' 'PT' 'LT' 'LU' 'LV' 'NL'
 'EE' 'FI' 'IE' 'SK' 'MT']


### 时间特征

In [15]:
df_exploded["year"] = df_exploded["publish_date"].dt.year
df_exploded["month"] = df_exploded["publish_date"].dt.month
df_exploded["month_ts"] = (
    df_exploded["publish_date"]
    .dt.to_period("M")
    .dt.to_timestamp("M")   # month end
)


### 主题flag

In [16]:
def make_flag(series: pd.Series, pattern: str) -> pd.Series:
    return series.fillna("").str.contains(pattern, case=False, regex=True).astype(int)

# manufacturing: anything containing "MANUFACTURING"
df_exploded["theme_manufacturing"] = make_flag(
    df_exploded["themes"], r"MANUFACTURING"
)

# industrial production
df_exploded["theme_industrial_prod"] = make_flag(
    df_exploded["themes"], r"INDUSTRIAL_PRODUCTION"
)

# energy: energy + oil/gas
df_exploded["theme_energy"] = make_flag(
    df_exploded["themes"], r"(ECON_ENERGY|ENV_ENERGY|ENERGY_POLICY|OIL|GAS)"
)

# macro policy
df_exploded["theme_policy"] = make_flag(
    df_exploded["themes"], r"(ECON_POLICY|ECON_MONETARY_POLICY|ECON_FISCAL|PUBLIC_FINANCE)"
)

# trade
df_exploded["theme_trade"] = make_flag(
    df_exploded["themes"], r"(ECON_TRADE|EXPORT|IMPORT|ECON_TARIFF)"
)


/tmp/ipykernel_47/2845810393.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return series.fillna("").str.contains(pattern, case=False, regex=True).astype(int)
/tmp/ipykernel_47/2845810393.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return series.fillna("").str.contains(pattern, case=False, regex=True).astype(int)
/tmp/ipykernel_47/2845810393.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return series.fillna("").str.contains(pattern, case=False, regex=True).astype(int)


### 从 URL 生成 “title”，并清洗

In [17]:
def title_from_url(url: str) -> str:
    if not isinstance(url, str):
        return ""
    text = re.sub(r"^https?://", "", url)
    text = text.split("?", 1)[0]
    slug = text.strip("/").split("/")[-1]
    slug = re.sub(r"\.(html?|php|aspx?)$", "", slug, flags=re.IGNORECASE)
    slug = slug.replace("-", " ").replace("_", " ")
    return slug


def clean_title(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.strip()
    t = re.sub(r"\s+", " ", t)

    t = re.sub(
        r"\s*[-–—]\s*(Reuters|Bloomberg|FT|Financial Times|The Guardian|NYTimes|New York Times|AP|AFP)$",
        "",
        t,
        flags=re.IGNORECASE,
    )
    t = re.sub(r"\s*\|\s*[^|]+$", "", t)
    return t


if "title" in df_exploded.columns:
    df_exploded["raw_title"] = df_exploded["title"]
else:
    df_exploded["raw_title"] = df_exploded["article_url"].apply(title_from_url)

df_exploded["clean_title"] = df_exploded["raw_title"].apply(clean_title)
df_exploded = df_exploded[df_exploded["clean_title"].str.len() >= 15].copy()

df_exploded["title_len_words"] = df_exploded["clean_title"].str.split().str.len()

print("Rows after title cleaning:", len(df_exploded))


Rows after title cleaning: 911015


In [18]:
import re
import urllib.parse

def refine_clean_title(text: str) -> str:
    if not isinstance(text, str):
        return ""

    t = text.strip()

    # URL decode: turn %C2%AE into ®, etc.
    t = urllib.parse.unquote(t)

    # remove long hex-like IDs
    t = re.sub(r"\b[0-9a-f]{8,}\b", "", t, flags=re.IGNORECASE)

    # remove leading generic words like "story", "article"
    t = re.sub(r"^(story|article)\s+", "", t, flags=re.IGNORECASE)

    # collapse whitespace
    t = re.sub(r"\s+", " ", t).strip()
    return t

df_exploded["clean_title"] = df_exploded["clean_title"].apply(refine_clean_title)

# drop very short titles after refinement
df_exploded = df_exploded[df_exploded["clean_title"].str.len() >= 10].copy()

# normalized title for dedup
df_exploded["clean_title_norm"] = (
    df_exploded["clean_title"].str.lower().str.strip()
)


### 在 国家×月份 内按标题去重

In [19]:
df_exploded = df_exploded.sort_values(
    ["country_iso2", "month_ts", "publish_time"]
)

df_exploded = df_exploded.drop_duplicates(
    subset=["country_iso2", "month_ts", "clean_title_norm"],
    keep="first",
).reset_index(drop=True)

print("Rows after dedup within country-month:", len(df_exploded))


Rows after dedup within country-month: 705846


In [20]:
# 1) drop rows whose themes clearly focus on entertainment people
entertainment_pattern = r"TAX_FNCACT_(ACTOR|ACTRESS|SINGER|MUSICIAN|COMEDIAN|FOOTBALL|BASKETBALL|BASEBALL)"
mask_ent_theme = df_exploded["themes"].str.contains(
    entertainment_pattern, na=False, case=False
)

# 2) drop rows from obvious entertainment sections in the URL
entertainment_url_pattern = r"(tvshowbiz|celebrity|entertainment|showbiz)"
mask_ent_url = df_exploded["article_url"].str.contains(
    entertainment_url_pattern, na=False, case=False
)

df_exploded = df_exploded[~(mask_ent_theme | mask_ent_url)].copy()
print("Rows after dropping entertainment-like noise:", len(df_exploded))


/tmp/ipykernel_47/1665304249.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_ent_theme = df_exploded["themes"].str.contains(
/tmp/ipykernel_47/1665304249.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_ent_url = df_exploded["article_url"].str.contains(


Rows after dropping entertainment-like noise: 665257


In [21]:
cols_to_drop = ["source_language", "translated_to_english"]
existing_to_drop = [c for c in cols_to_drop if c in df_exploded.columns]

df_exploded = df_exploded.drop(columns=existing_to_drop)


In [22]:
df_exploded["tone_main_clipped"] = df_exploded["tone_main"].clip(-10, 10)

df_exploded["tone_z_within_country"] = (
    df_exploded.groupby("country_iso2")["tone_main_clipped"]
    .transform(lambda s: (s - s.mean()) / s.std(ddof=0))
)

df_exploded.to_parquet("euro_manu_news_clean_exploded.parquet", index=False)
print("Final cleaned exploded shape:", df_exploded.shape)


Final cleaned exploded shape: (665257, 26)


In [23]:
# sample a small subset for quick preview in Excel / Sheets
output_sample = "euro_manu_news_clean_exploded_sample_1000.csv"

sample_df = (
    df_exploded
    .sample(n=min(1000, len(df_exploded)), random_state=0)
    .sort_values(["country_iso2", "publish_date"])
)

sample_df.to_csv(output_sample, index=False)
print("Saved preview sample CSV to:", output_sample)
print("Sample shape:", sample_df.shape)


Saved preview sample CSV to: euro_manu_news_clean_exploded_sample_1000.csv
Sample shape: (1000, 26)


In [24]:
import numpy as np

panel = (
    df_exploded.groupby(["country_iso2", "month_ts"])
      .agg(
          news_count=("article_url", "count"),
          tone_mean=("tone_main_clipped", "mean"),
          tone_std=("tone_main_clipped", "std"),
          tone_min=("tone_main_clipped", "min"),
          tone_max=("tone_main_clipped", "max"),
          avg_title_len_words=("title_len_words", "mean"),
          avg_location_count=("location_count", "mean"),
          share_theme_manufacturing=("theme_manufacturing", "mean"),
          share_theme_industrial_prod=("theme_industrial_prod", "mean"),
          share_theme_energy=("theme_energy", "mean"),
          share_theme_policy=("theme_policy", "mean"),
          share_theme_trade=("theme_trade", "mean"),
      )
      .reset_index()
      .sort_values(["country_iso2", "month_ts"])
)

# derived features
panel["news_log_count"] = np.log1p(panel["news_count"])
panel["tone_iqr_proxy"] = panel["tone_max"] - panel["tone_min"]

panel.to_csv("panel_euro_manu_tone_enhanced.csv", index=False)
print("Panel shape:", panel.shape)


Panel shape: (2470, 16)
